# **UNIVERSIDAD TORCUATO DI TELLA**
## **MAESTRÍA EN ECONOMETRÍA**

---

### **TRABAJO PRÁCTICO DE ECONOMETRÍA**

- **Profesor:** González-Rozada, Martín  
- **Ayudante:** Lening, Iara  
- **Alumno:** Guzzi, David Alexander  (Legajo n°: 24H1970)  

**Ciclo Lectivo:** Tercer Trimestre, 2024  

---

##### **1. PROPIEDADES DE MUESTRA FINITA DE FGLS (MCGE).**

In [36]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

np.random.seed(144)
n_reps = 5000
n_obs = 500

# Función que ejecuta el procedimiento una vez
def run_simulation():
    x = np.random.uniform(1, 10, n_obs)
    x2 = x ** 2
    u = np.random.normal(scale=x)
    
    y0 = 1 + x + u
    y1 = 1 + 0.5 * x + u
    y2 = 1 + u
    
    def fgls(y, x):
        ols = sm.OLS(y, sm.add_constant(x)).fit()
        resid2 = ols.resid ** 2
        gamma_hat = sm.OLS(resid2, x2).fit().params[0]
        
        uhat2_hat = gamma_hat * x2
        weights = np.sqrt(uhat2_hat)
        x_gls = x / weights
        y_gls = y / weights
        cons_gls = 1 / weights
        gls = sm.OLS(y_gls, np.column_stack((cons_gls, x_gls))).fit()
        
        p_value = gls.pvalues[1]  # Para la prueba H0: beta = 1
        return p_value, gls.params[0], gls.params[1], gamma_hat
    
    pv0, beta0, beta1, gamma_hat = fgls(y0, x)
    pv1, _, _, _ = fgls(y1, x)
    pv2, _, _, _ = fgls(y2, x)
    
    return pv0, pv1, pv2, beta0, beta1, gamma_hat

# Ejecutamos la simulación
results = np.array([run_simulation() for _ in range(n_reps)])

# Extraemos los resultados
pv0, pv1, pv2, beta0_vals, beta1_vals, gamma_hat_vals = results.T

# Calculamos los tamaños del test y la potencia
pv0_5 = np.mean(pv0 < 0.05)
pv0_1 = np.mean(pv0 < 0.01)
pv1_5 = np.mean(pv1 < 0.05)
pv1_1 = np.mean(pv1 < 0.01)
pv2_5 = np.mean(pv2 < 0.05)
pv2_1 = np.mean(pv2 < 0.01)

# Mostramos los resultados
print(f"Con un nivel de significatividad del 5%, el tamaño del test es {pv0_5:.4f}")
print(f"Con un nivel de significatividad del 1%, el tamaño del test es {pv0_1:.4f}")
print(f"Con un nivel de significatividad del 5%, cuando el valor verdadero de b es 0.5, la potencia del test es {pv1_5:.4f}")
print(f"Con un nivel de significatividad del 1%, cuando el valor verdadero de b es 0.5, la potencia del test es {pv1_1:.4f}")
print(f"Con un nivel de significatividad del 5%, cuando el valor verdadero de b es 0, la potencia del test es {pv2_5:.4f}")
print(f"Con un nivel de significatividad del 1%, cuando el valor verdadero de b es 0, la potencia del test es {pv2_1:.4f}")

# Estadísticas de resumen
summary_df = pd.DataFrame({
    'beta0': beta0_vals,
    'beta1': beta1_vals,
    'gamma_hat': gamma_hat_vals
})
print(summary_df.describe(percentiles=[0.5]))

Con un nivel de significatividad del 5%, el tamaño del test es 1.0000
Con un nivel de significatividad del 1%, el tamaño del test es 1.0000
Con un nivel de significatividad del 5%, cuando el valor verdadero de b es 0.5, la potencia del test es 1.0000
Con un nivel de significatividad del 1%, cuando el valor verdadero de b es 0.5, la potencia del test es 1.0000
Con un nivel de significatividad del 5%, cuando el valor verdadero de b es 0, la potencia del test es 0.0522
Con un nivel de significatividad del 1%, cuando el valor verdadero de b es 0, la potencia del test es 0.0118
             beta0        beta1    gamma_hat
count  5000.000000  5000.000000  5000.000000
mean      0.994148     1.001113     0.992565
std       0.244381     0.076791     0.098951
min      -0.034371     0.728956     0.670566
50%       0.989151     1.003207     0.986854
max       1.899646     1.343259     1.406991


In [37]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Configuración inicial
np.random.seed(3649)

# Función para realizar FGLS
def fgls_estimation(x, y):
    # DataFrame para almacenar las variables intermedias
    df = pd.DataFrame({'x': x, 'y': y})
    df['x2'] = df['x'] ** 2

    # 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    ols_model = sm.OLS(y, X)  # Estimación OLS
    ols_results = ols_model.fit()  # Ajuste de OLS

    # 2. Calcular los residuos del modelo y elevarlos al cuadrado
    df['u_hat'] = ols_results.resid
    df['u_hat2'] = df['u_hat']**2

    # 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x + x2, se estima por OLS usando u_hat2 como proxy de sigma2
    aux_X = sm.add_constant(df[['x', 'x2']])
    ols_model_aux = sm.OLS(df['u_hat2'], aux_X)
    ols_model_aux_results = ols_model_aux.fit()
    gamma_hat1 = ols_model_aux_results.params['x']
    gamma_hat2 = ols_model_aux_results.params['x2']

    # 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes)
    df['sigma2_hat'] = ols_model_aux_results.predict(aux_X)

    # 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por sigma2_hat y estimar por OLS
    df['u_hat2_over_sigma2_hat'] = df['u_hat2'] / df['sigma2_hat']
    df['const1'] = 1 / df['sigma2_hat']
    df['x_over_sigma2_hat'] = df['x'] / df['sigma2_hat']
    df['x2_over_sigma2_hat'] = df['x2'] / df['sigma2_hat']
    

    # Crear la matriz de diseño constante, x_over_sigma2_hat y x2_over_sigma2_hat
    aux_X_2 = df[['const1', 'x_over_sigma2_hat', 'x2_over_sigma2_hat']]

    ols_model_aux2 = sm.OLS(df['u_hat2_over_sigma2_hat'], aux_X_2)
    ols_model_aux_results2 = ols_model_aux2.fit()
    gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
    gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

    # 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
    df['sigma2_tilde'] = ols_model_aux_results2.predict(aux_X_2)
    
    # Corregir valores negativos o muy pequeños
    df['sigma2_tilde'] = df['sigma2_tilde'].clip(lower=1e-10)
    df['sigma_tilde'] = np.sqrt(df['sigma2_tilde'])

    # Verificar si hay valores NaN
    if df['sigma_tilde'].isna().any():
        raise ValueError("Se encontraron valores NaN en sigma_tilde")

    # 7. Usar uno sobre sigma tilde como ponderador en la regresión de y ~ x
    df['y_star'] = df['y'] / df['sigma_tilde']
    df['const2'] = 1 / df['sigma_tilde']
    df['x_star'] = df['x'] / df['sigma_tilde']

    # Crear la matriz de diseño con constante y x_star
    aux_X_star = df[['const2', 'x_star']]

    # Ajustar el modelo OLS
    final_ols_model = sm.OLS(df['y_star'], aux_X_star)
    final_ols_results = final_ols_model.fit() 

    # Estimaciones finales
    beta_0_hat = final_ols_results.params['const2']
    beta_1_hat = final_ols_results.params['x_star']
    se_beta1_hat = final_ols_results.bse.iloc[1]

    return beta_0_hat, beta_1_hat, se_beta1_hat

In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Configuración inicial
np.random.seed(3649)

# Función para realizar FGLS
def fgls_estimation2(x, y):
    # DataFrame para almacenar las variables intermedias
    df = pd.DataFrame({'x': x, 'y': y})
    df['x2'] = df['x'] ** 2

    # 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    ols_model = sm.OLS(y, X)  # Estimación OLS
    ols_results = ols_model.fit()  # Ajuste de OLS

    # 2. Calcular los residuos del modelo y elevarlos al cuadrado
    df['u_hat'] = ols_results.resid
    df['u_hat2'] = df['u_hat']**2

    # 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x + x2, se estima por OLS usando u_hat2 como proxy de sigma2
    aux_X = sm.add_constant(df[['x', 'x2']])
    ols_model_aux = sm.OLS(df['u_hat2'], aux_X)
    ols_model_aux_results = ols_model_aux.fit()
    gamma_hat1 = ols_model_aux_results.params['x']
    gamma_hat2 = ols_model_aux_results.params['x2']

    # 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes)
    df['sigma2_hat'] = ols_model_aux_results.predict(aux_X).clip(lower=1e-10)
    df['sigma_hat'] = np.sqrt(df['sigma2_hat'])

    # 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por sigma2_hat y estimar por OLS
    df['y_over_sigma2_hat'] = df['y'] / df['sigma_hat']
    df['const1'] = 1 / df['sigma_hat']
    df['x_over_sigma2_hat'] = df['x'] / df['sigma_hat']
    # df['x2_over_sigma2_hat'] = df['x2'] / df['sigma_hat']
    

    # Crear la matriz de diseño constante, x_over_sigma2_hat y x2_over_sigma2_hat
    aux_X_2 = df[['const1', 'x_over_sigma2_hat']]

    ols_model_aux2 = sm.OLS(df['y_over_sigma2_hat'], aux_X_2)
    ols_model_aux_results2 = ols_model_aux2.fit()
    # gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
    # gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

    # Estimaciones finales
    beta_0_hat = ols_model_aux_results2.params['const1']
    beta_1_hat = ols_model_aux_results2.params['x_over_sigma2_hat']
    se_beta1_hat = ols_model_aux_results2.bse.iloc[1]

    # # 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
    # df['sigma2_tilde'] = ols_model_aux_results2.predict(aux_X_2)
    
    # # Corregir valores negativos o muy pequeños
    # df['sigma2_tilde'] = df['sigma2_tilde'].clip(lower=1e-10)
    # df['sigma_tilde'] = np.sqrt(df['sigma2_tilde'])

    # # Verificar si hay valores NaN
    # if df['sigma_tilde'].isna().any():
    #     raise ValueError("Se encontraron valores NaN en sigma_tilde")

    # # 7. Usar uno sobre sigma tilde como ponderador en la regresión de y ~ x
    # df['y_star'] = df['y'] / df['sigma_tilde']
    # df['const2'] = 1 / df['sigma_tilde']
    # df['x_star'] = df['x'] / df['sigma_tilde']

    # # Crear la matriz de diseño con constante y x_star
    # aux_X_star = df[['const2', 'x_star']]

    # # Ajustar el modelo OLS
    # final_ols_model = sm.OLS(df['y_star'], aux_X_star)
    # final_ols_results = final_ols_model.fit() 

    # # Estimaciones finales
    # beta_0_hat = final_ols_results.params['const2']
    # beta_1_hat = final_ols_results.params['x_star']
    # se_beta1_hat = final_ols_results.bse.iloc[1]

    return beta_0_hat, beta_1_hat, se_beta1_hat

In [27]:
def fgls_estimation_numpy(x, y):
    # 1. Estimación inicial por OLS
    X = np.column_stack((np.ones_like(x), x))  # Agregar intercepto
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y  # Estimación OLS
    u_hat = y - X @ beta_ols  # Residuos
    u_hat2 = u_hat ** 2  # Cuadrado de los residuos
    
    # 2. Regresión auxiliar para estimar la heterocedasticidad
    X_aux = np.column_stack((np.ones_like(x), x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ u_hat2
    sigma2_hat = np.maximum(X_aux @ gamma_hat, 1e-10)  # Asegurar positividad
    sigma_hat = np.sqrt(sigma2_hat)
    
    # 3. Transformación de variables y segunda estimación por OLS
    y_trans = y / sigma_hat
    X_trans = np.column_stack((np.ones_like(x) / sigma_hat, x / sigma_hat))
    beta_fgls = np.linalg.inv(X_trans.T @ X_trans) @ X_trans.T @ y_trans
    
    # 4. Cálculo de errores estándar
    residuals_fgls = y_trans - X_trans @ beta_fgls
    sigma2_fgls = np.sum(residuals_fgls**2) / (len(x) - X_trans.shape[1])
    var_beta_fgls = sigma2_fgls * np.linalg.inv(X_trans.T @ X_trans)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Configuración inicial
np.random.seed(3649)

def fgls_estimation_numpy2(x, y):
    # 1. Estimación inicial por OLS
    X = np.column_stack((np.ones_like(x), x))  # Agregar intercepto
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y  # Estimación OLS
    u_hat = y - X @ beta_ols  # Residuos
    u_hat2 = u_hat ** 2  # Cuadrado de los residuos
    
    # 2. Regresión auxiliar para estimar la heterocedasticidad
    X_aux = np.column_stack((np.ones_like(x), x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ u_hat2
    sigma2_hat = X_aux @ gamma_hat

    X_aux2 = np.column_stack((np.ones_like(x) / sigma2_hat, x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    sigma2_tilde = X_aux2 @ gamma_hat2  # Asegurar positividad
    sigma_tilde = np.sqrt(sigma2_tilde)
    
    # 3. Transformación de variables y segunda estimación por OLS
    y_trans = y / sigma_tilde
    X_trans = np.column_stack((np.ones_like(x) / sigma_tilde, x / sigma_tilde))
    beta_fgls = np.linalg.inv(X_trans.T @ X_trans) @ X_trans.T @ y_trans
    
    # 4. Cálculo de errores estándar
    residuals_fgls = y_trans - X_trans @ beta_fgls
    sigma2_fgls = np.sum(residuals_fgls**2) / (len(x) - X_trans.shape[1])
    var_beta_fgls = sigma2_fgls * np.linalg.inv(X_trans.T @ X_trans)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Configuración inicial
np.random.seed(3649)

def fgls_estimation_numpy3(x, y):
    # 1. Estimación inicial por OLS
    X = np.column_stack((np.ones_like(x), x))  # Agregar intercepto
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y  # Estimación OLS
    u_hat = y - X @ beta_ols  # Residuos
    u_hat2 = u_hat ** 2  # Cuadrado de los residuos
    
    # 2. Regresión auxiliar para estimar la heterocedasticidad
    X_aux = np.column_stack((np.ones_like(x), x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ u_hat2
    sigma2_hat = X_aux @ gamma_hat

    X_aux2 = np.column_stack((np.ones_like(x) / sigma2_hat, x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    sigma2_tilde = X_aux @ gamma_hat2  # Asegurar positividad
    # sigma_tilde = np.sqrt(sigma2_tilde)
    omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde))
    
    # # 3. Transformación de variables y segunda estimación por OLS
    # y_trans = y / sigma_tilde
    # X_trans = np.column_stack((np.ones_like(x) / sigma_tilde, x / sigma_tilde))
    beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y
    
    # 4. Cálculo de errores estándar
    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / (len(x) - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]   
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Configuración inicial
np.random.seed(3649)

def fgls_estimation_numpy4(x, y):
    
    X = np.column_stack((np.ones_like(x), x)) 
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y 
    u_hat = y - X @ beta_ols
    u_hat2 = u_hat ** 2  
    log_u_hat2 = np.log(u_hat2)
    
    X_aux = np.column_stack((x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
    log_sigma2_hat = X_aux @ gamma_hat
    sigma2_hat = np.exp(log_sigma2_hat)

    X_aux2 = np.column_stack((x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    sigma2_tilde = X_aux @ gamma_hat2 
    sigma2_tilde = np.maximum(sigma2_tilde, 1e-6)
    omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde))
    
    
    beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y
    
    # 4. Cálculo de errores estándar
    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / ( X.shape[0] - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]   
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [ ]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm, t, chi2

np.random.seed(3649)


n_obs = 500
beta_0_true = -3
beta_1_true = 0.8

x = np.random.uniform(1, 50, n_obs)
u = np.random.normal(scale=x)
y = beta_0_true + beta_1_true * x + u

X = np.column_stack((np.ones_like(x), x)) 
beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y 
u_hat = y - X @ beta_ols
u_hat2 = u_hat ** 2  
log_u_hat2 = np.log(u_hat2)

print(f"u_hat2: {u_hat2}, log_u_hat2:{log_u_hat2}")


X_aux = np.column_stack((np.ones_like(x), x, x**2))
gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
log_sigma2_hat = X_aux @ gamma_hat
sigma2_hat = np.exp(log_sigma2_hat)

print(f"log_sigma2_hat:{log_sigma2_hat}, sigma2_hat:{sigma2_hat}")

X_aux2 = np.column_stack((np.ones_like(x) / sigma2_hat, x / sigma2_hat,  x**2 / sigma2_hat))
u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

print(f"gamma_hat2:{gamma_hat2}")

sigma2_tilde = X_aux @ gamma_hat2 
sigma2_tilde = np.maximum(sigma2_tilde, 1e-6)
omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde))

print(f"sigma2_tilde: {sigma2_tilde}, omega_tilde_inv:{omega_tilde_inv}")

beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y

print(f"beta_fgls: {beta_fgls}")

residuals_fgls = y - X @ beta_fgls
s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / ( X.shape[0] - X.shape[1])
var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]

print(f"se_beta1_fgls:{se_beta1_fgls}")

t_stat = (beta_fgls[1] - 0.8) / se_beta1_fgls
tp_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
np_value = 2 * (1 - norm.cdf(abs(t_stat)))
chip_value = 1 - chi2.cdf(t_stat, df=1)

print(f"t_stat: {t_stat}, tp_value:{tp_value}, np_value:{np_value}, chip_value:{chip_value}")


# Ajustar el modelo GLS (equivalente a WLS si Omega es diagonal)
model = sm.GLS(y, X, sigma=sigma2_tilde).fit()  # sigma es la inversa de los pesos

# Extraer coeficientes y errores estándar
beta_0, beta_1 = model.params
se_beta1 = model.bse[1]
pval_beta1 = model.pvalues[1]
hypothesis = 'x1 = 0.8'  # Aquí 'x1' es el nombre de la variable correspondiente a beta1 en el modelo

# Realizar la prueba t
t_test = model.t_test(hypothesis)

# Obtener el estadístico t y el p-valor
t_statistic = t_test.tvalue[0][0]
p_value_res = t_test.pvalue

print(f"beta_0 = {beta_0}, beta_1 = {beta_1}, se_beta1 = {se_beta1}")

print(f"pval_beta1 = {pval_beta1}, t_test_res = {p_value_res}, t_statistic = {t_statistic}")

u_hat2: [3.72274068e+03 4.93029693e+02 1.76274518e+03 8.95859030e+02
 1.20397116e+03 1.45748562e+01 1.71183514e+00 3.68287155e+01
 4.80798104e+01 7.16184304e+02 1.81838223e+02 1.75854284e+01
 4.21535431e+02 2.78471371e+02 1.56941745e+01 3.68948185e+01
 1.26841166e+03 2.42232780e+02 4.98376119e-02 1.07450608e+02
 9.47909850e+02 1.19316035e+00 3.75589602e+02 7.94265394e+01
 5.36467879e+02 8.97070726e+01 4.95216021e+00 5.98619171e+01
 6.65142815e+01 8.83710516e+01 1.33054545e+01 5.32474385e-01
 1.07664898e+03 1.12914411e+03 1.23142937e+02 2.32139660e-02
 9.08496571e+00 1.48158691e+02 2.92085565e+03 4.44394536e+01
 1.61321095e+01 1.22198614e+03 3.16575760e+00 1.20380979e+01
 2.43842096e+03 2.24704849e+03 7.79691426e+02 1.03845107e+01
 4.14678838e+02 2.66252607e-02 3.54182652e+03 1.36365565e+01
 4.09515181e+01 9.91182818e+01 1.96331907e+02 3.55326325e+03
 3.59422321e+02 5.33557790e+01 1.06220202e+01 1.88030655e+01
 9.85238312e+01 6.52230599e+02 1.06748002e+01 9.11034282e+02
 1.23823306e+03 

In [51]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import t

np.random.seed(3649)

reject_1pct_list = []
reject_5pct_list = []


for i in range(50):
    n_obs = 10
    beta_0_true = -3
    beta_1_true = 0.8

    x = np.random.uniform(1, 50, n_obs)
    u = np.random.normal(scale=x)
    y = beta_0_true + beta_1_true * x + u

    X = np.column_stack((np.ones_like(x), x)) 
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y 
    u_hat = y - X @ beta_ols
    u_hat2 = u_hat ** 2  
    log_u_hat2 = np.log(u_hat2)

    X_aux = np.column_stack((np.ones_like(x), x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
    log_sigma2_hat = X_aux @ gamma_hat
    sigma2_hat = np.exp(log_sigma2_hat)

    X_aux2 = np.column_stack((np.ones_like(x) / sigma2_hat, x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    sigma2_tilde = X_aux @ gamma_hat2 
    sigma2_tilde = np.maximum(sigma2_tilde, 1e-6)
    omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde))

    beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y

    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / ( X.shape[0] - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]

    t_stat = (beta_fgls[1] - 0.8) / se_beta1_fgls
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
    reject_1pct = p_value < 0.01
    reject_5pct = p_value < 0.05

    reject_1pct_list.append(reject_1pct)
    reject_5pct_list.append(reject_5pct)

print(f"reject_1pct_list: {np.mean(reject_1pct_list)}")
print(f"reject_5pct_list: {np.mean(reject_5pct_list)}")

reject_1pct_list: 0.04
reject_5pct_list: 0.12


In [61]:
import numpy as np
import statsmodels.api as sm

# Datos definidos manualmente
X_values = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 50])
y_values = np.array([2.5, 70.1, 3.8, 4.5, 5.2, 5.9, 6.5, 7.2, 7.8, 8.5])

# Agregar constante
X = sm.add_constant(X_values)

# Ajustar modelo OLS
model = sm.OLS(y_values, X)
results = model.fit()

hypothesis = 'x1 = 0.8'  # Aquí 'x1' es el nombre de la variable correspondiente a beta1 en el modelo

# Realizar la prueba t
t_test = results.t_test(hypothesis)

# Obtener el estadístico t y el p-valor
t_statistic = t_test.tvalue[0][0]
p_value_res = t_test.pvalue

print(f"P-valor OLS (Python): {p_value_res:.6f}")


P-valor OLS (Python): 0.084999


In [63]:
from scipy.stats import f
# Obtener estimaciones
beta_1_hat = results.params[1]          # Coeficiente estimado
se_beta1 = results.bse[1]                # Error estándar de beta_1
n_obs = len(y_values)                     # Número de observaciones
df_num = 1                                 # Grados de libertad del numerador
df_den = n_obs - 2                         # Grados de libertad del denominador

# Estadístico F
H0 = 1  # Hipótesis nula beta_1 = 1
F_stat = ((beta_1_hat - H0) ** 2) / (se_beta1 ** 2)

# P-valor usando distribución F
p_value_F = 1 - f.cdf(F_stat, df_num, df_den)

print(f"Estadístico F: {F_stat:.6f}")
print(f"P-valor (distribución F): {p_value_F:.6f}")

Estadístico F: 5.610111
P-valor (distribución F): 0.045348


In [66]:
def fgls_estimation_numpy5(x, y):
    
    X = np.column_stack((np.ones_like(x), x)) 
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y 
    u_hat = y - X @ beta_ols
    u_hat2 = u_hat ** 2  
    log_u_hat2 = np.log(np.maximum(u_hat2, 1e-6))  # Evita log(0)
    
    # Estimación de la varianza condicional con un modelo cuadrático
    X_aux = np.column_stack((np.ones_like(x), x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
    log_sigma2_hat = X_aux @ gamma_hat
    sigma2_hat = np.exp(log_sigma2_hat)

    # Asegurar que sigma2_hat no sea cero ni negativo
    sigma2_hat = np.maximum(sigma2_hat, 1e-6)

    # Matriz de pesos (diagonal)
    omega_inv = np.diag(1 / sigma2_hat)

    # Estimación FGLS
    beta_fgls = np.linalg.inv(X.T @ omega_inv @ X) @ X.T @ omega_inv @ y

    # Cálculo de errores estándar
    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_inv @ residuals_fgls) / (len(x) - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]   

    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [89]:
np.random.seed(3649)
def simulate_fgls(n_samples, n_observations_list, beta_0_true, beta_1_true, beta1_H0):
    
    """Realiza simulaciones de FGLS para distintos tamaños de muestra y reporta estadísticas de interés."""
    
    results = []
    for n_obs in n_observations_list:
        sample_results = []
        
        for i in range(n_samples):
            # Generación de datos
            x = np.random.uniform(1, 50, n_obs)
            u = np.random.normal(scale=x)
            y = beta_0_true + beta_1_true * x + u
            
            # Estimación FGLS
            beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation_numpy4(x, y)
            
            # Test de hipótesis para beta_1 = 0.8
            t_stat = (beta_1_hat - beta1_H0) / se_beta1_hat
            p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
            reject_1pct = p_value < 0.01
            reject_5pct = p_value < 0.05
            
            sample_results.append([beta_0_hat, beta_1_hat, reject_1pct, reject_5pct])
        
        # Convertir a DataFrame
        df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])
        
        # Calcular estadísticas
        mean_beta0 = df_results['beta_0_hat'].mean()
        mean_beta1 = df_results['beta_1_hat'].mean()
        median_beta0 = df_results['beta_0_hat'].median()
        median_beta1 = df_results['beta_1_hat'].median()
        std_beta0 = df_results['beta_0_hat'].std()
        std_beta1 = df_results['beta_1_hat'].std()
        test_size_1pct = df_results['reject_1pct'].mean()
        test_size_5pct = df_results['reject_5pct'].mean()
        
        results.append([n_obs, mean_beta0, median_beta0, std_beta0, mean_beta1, median_beta1, std_beta1, test_size_1pct, test_size_5pct])
    
    # Crear DataFrame final
    results_df = pd.DataFrame(results, columns=['n_observations', 'mean_beta0', 'median_beta0', 'std_beta0', 'mean_beta1', 'median_beta1', 'std_beta1', 'test_size_1pct', 'test_size_5pct'])
    
    return results_df

# Parámetros iniciales
test_results = simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500, 700, 1000, 1200], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8)
test_results

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_size_1pct,test_size_5pct
0,5,9.258171,-2.623278,893.446389,0.459207,0.786858,22.237986,0.1124,0.2204
1,10,18.481732,-2.940235,994.586445,0.314504,0.795749,23.291711,0.1118,0.1968
2,30,-14.900211,-2.985833,630.791366,1.004283,0.801668,13.521860,0.0750,0.1512
3,100,-4.380422,-3.001054,88.507895,0.839335,0.800465,3.178552,0.0578,0.1088
4,200,-2.960183,-2.996994,8.543328,0.805510,0.799963,2.897765,0.0480,0.0952
5,500,-2.996185,-2.979176,2.097564,0.809875,0.798362,1.788327,0.0370,0.0768
6,700,-3.019977,-3.011490,1.855264,0.811589,0.799311,1.683692,0.0292,0.0690
7,1000,-3.005423,-2.993798,1.609359,0.808850,0.799982,1.456669,0.0236,0.0606
8,1200,-2.985904,-2.994522,1.102379,0.791900,0.800001,0.993060,0.0166,0.0584


In [3]:
np.random.seed(3649)
def simulate_fgls(n_samples, n_observations_list, beta_0_true, beta_1_true, beta1_H0):
    
    """Realiza simulaciones de FGLS para distintos tamaños de muestra y reporta estadísticas de interés."""
    
    results = []
    for n_obs in n_observations_list:
        sample_results = []
        
        for i in range(n_samples):
            # Generación de datos
            x = np.random.uniform(1, 50, n_obs)
            u = np.random.normal(scale=x)
            y = beta_0_true + beta_1_true * x + u
            
            # Estimación FGLS
            beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation_numpy4(x, y)
            
            # Test de hipótesis para beta_1 = 0.8
            t_stat = (beta_1_hat - beta1_H0) / se_beta1_hat
            p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
            reject_1pct = p_value < 0.01
            reject_5pct = p_value < 0.05
            
            sample_results.append([beta_0_hat, beta_1_hat, reject_1pct, reject_5pct])
        
        # Convertir a DataFrame
        df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])
        
        # Calcular estadísticas
        mean_beta0 = df_results['beta_0_hat'].mean()
        mean_beta1 = df_results['beta_1_hat'].mean()
        median_beta0 = df_results['beta_0_hat'].median()
        median_beta1 = df_results['beta_1_hat'].median()
        std_beta0 = df_results['beta_0_hat'].std()
        std_beta1 = df_results['beta_1_hat'].std()
        test_size_1pct = df_results['reject_1pct'].mean()
        test_size_5pct = df_results['reject_5pct'].mean()
        
        results.append([n_obs, mean_beta0, median_beta0, std_beta0, mean_beta1, median_beta1, std_beta1, test_size_1pct, test_size_5pct])
    
    # Crear DataFrame final
    results_df = pd.DataFrame(results, columns=['n_observations', 'mean_beta0', 'median_beta0', 'std_beta0', 'mean_beta1', 'median_beta1', 'std_beta1', 'test_size_1pct', 'test_size_5pct'])
    
    return results_df
test_results = simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200], beta_0_true=-3, beta_1_true=0.4, beta1_H0=0.8)
test_results

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_size_1pct,test_size_5pct
0,5,9.258171,-2.623278,893.446389,0.059207,0.386858,22.237986,0.1230,0.2364
1,10,18.481732,-2.940235,994.586445,-0.085496,0.395749,23.291711,0.1356,0.2486
2,30,-14.900211,-2.985833,630.791366,0.604283,0.401668,13.521860,0.2002,0.3870
3,100,-4.380422,-3.001054,88.507895,0.439335,0.400465,3.178552,0.6858,0.8460
4,200,-2.960183,-2.996994,8.543328,0.405510,0.399963,2.897765,0.9572,0.9864


In [77]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import t

# Parámetros
n_obs = 500  # Número de observaciones por simulación
n_sim = 5000  # Número de simulaciones
beta_0_true = -3
beta_1_true = 0.8
sigma = 1  # Desviación estándar del error

# Almacenar resultados
beta1_estimates = np.zeros(n_sim)
se_beta1_estimates = np.zeros(n_sim)
p_values = np.zeros(n_sim)

for i in range(n_sim):
    # Generar datos
    x = np.random.uniform(0, 10, n_obs)
    epsilon = np.random.normal(0, sigma, n_obs)
    y = beta_0_true + beta_1_true * x + epsilon
    
    # Agregar constante para el intercepto
    X = sm.add_constant(x)
    
    # Ajustar modelo OLS
    model = sm.OLS(y, X).fit()
    
    # Extraer beta1, su error estándar y calcular p-valor
    beta1_estimates[i] = model.params[1]
    se_beta1_estimates[i] = model.bse[1]
    t_stat = (model.params[1] - 0.8) / model.bse[1]
    p_values[i] = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))

# Resultados promedio
mean_beta1 = np.mean(beta1_estimates)
mean_se_beta1 = np.mean(se_beta1_estimates)
mean_p_value = np.mean(p_values)

print(f"Promedio Beta1: {mean_beta1:.4f}")
print(f"Promedio SE Beta1: {mean_se_beta1:.4f}")
print(f"Proporción de rechazos H0 (p<0.05): {np.mean(p_values < 0.01):.4f}")

Promedio Beta1: 0.7996
Promedio SE Beta1: 0.0155
Proporción de rechazos H0 (p<0.05): 0.0106


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df_results["beta_1_hat"], kde=True)
plt.axvline(beta1_H0, color='red', linestyle="--", label="Valor de H0: 0.8")
plt.legend()
plt.title("Distribución de Beta 1 Estimado")
plt.show()

KeyError: 'beta_1_hat'

In [7]:
def simulate_fgls(n_samples, n_observations_list, beta_0_true, beta_1_true, beta1_H0):
    
    """Realiza simulaciones de FGLS para distintos tamaños de muestra y reporta estadísticas de interés."""
    
    results = []
    for n_obs in n_observations_list:
        sample_results = []
        
        for i in range(n_samples):
            # Generación de datos
            x = np.random.uniform(1, 50, n_obs)
            u = np.random.normal(scale=x)
            y = beta_0_true + beta_1_true * x + u
            
            # Estimación FGLS
            beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation(x, y)
            
            # Test de hipótesis para beta_1 = 0.8
            t_stat = (beta_1_hat - beta1_H0) / se_beta1_hat
            p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
            reject_1pct = p_value < 0.01
            reject_5pct = p_value < 0.05
            
            sample_results.append([beta_0_hat, beta_1_hat, reject_1pct, reject_5pct])
        
        # Convertir a DataFrame
        df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])
        
        # Calcular estadísticas
        mean_beta0 = df_results['beta_0_hat'].mean()
        mean_beta1 = df_results['beta_1_hat'].mean()
        median_beta0 = df_results['beta_0_hat'].median()
        median_beta1 = df_results['beta_1_hat'].median()
        std_beta0 = df_results['beta_0_hat'].std()
        std_beta1 = df_results['beta_1_hat'].std()
        test_size_1pct = df_results['reject_1pct'].mean()
        test_size_5pct = df_results['reject_5pct'].mean()
        
        results.append([n_obs, mean_beta0, median_beta0, std_beta0, mean_beta1, median_beta1, std_beta1, test_size_1pct, test_size_5pct])
    
    # Crear DataFrame final
    results_df = pd.DataFrame(results, columns=['n_observations', 'mean_beta0', 'median_beta0', 'std_beta0', 'mean_beta1', 'median_beta1', 'std_beta1', 'test_size_1pct', 'test_size_5pct'])
    
    return results_df

# Parámetros iniciales
test_results = simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.4, beta1_H0=0.8)
test_results

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_size_1pct,test_size_5pct
0,5,7.886475,-3.021852,811.247071,-0.161186,0.438020,44.853066,0.0710,0.1600
1,10,-4.736752,-2.840837,716.166070,-0.040182,0.379416,48.722276,0.2140,0.3140
2,30,-2.923546,-3.069659,162.486675,0.118178,0.391275,34.192519,0.3958,0.5120
3,100,-3.011968,-2.999841,134.097435,0.240242,0.399580,26.483190,0.5702,0.6856
4,200,-2.114750,-2.927595,86.312413,0.120036,0.393478,17.257694,0.6822,0.7818
5,500,-2.666872,-2.989617,62.486918,0.329250,0.396693,12.473935,0.8006,0.8490


In [ ]:
import numpy as np
import pandas as pd

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
n_observations = 5  # Observaciones por muestra
n_samples = 5000  # Número de muestras

# Matriz omega (varianzas)
omega = np.diag([4, 9, 16, 25, 36])

# Descomposición de Cholesky de omega
P = np.linalg.cholesky(omega)

# Inversa de P
P_inv = np.linalg.inv(P)

# Contenedores para almacenar muestras y estimadores
samples = []
results = []

# Simulación de M muestras
for sample_id in range(n_samples):
    # 1. Generar datos
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0_true + beta_1_true * x + u  # Generar y

    # Almacenar muestra
    samples.append(pd.DataFrame({'sample': i, 'x': x, 'y': y, 'u': u}))

    # 2. Transformar para MCG
    y_tilde = P_inv @ y
    X = np.column_stack((np.ones(n_observations), x))  # Matriz de diseño original
    X_tilde = P_inv @ X

    # 3. Ajustar modelo transformado con MCO
    beta_hat = np.linalg.inv(X_tilde.T @ X_tilde) @ (X_tilde.T @ y_tilde)

    # Almacenar resultados
    results.append([i, beta_0_hat, beta_1_hat, se_beta1_hat])

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples).reset_index(drop=True)

# Crear DataFrame con los resultados de FGLS
results_df = pd.DataFrame(results, columns=[
    'sample', 'beta_0_hat', 'beta_1_hat', 'se_beta1_hat'
])

# Mostrar ejemplos de los DataFrames generados
print("Ejemplo de all_samples:")
print(all_samples.head())
print("\nEjemplo de results_df:")
print(results_df.head())

In [ ]:
#Analizar.
def fgls_estimation(x, y):
    """Función de estimación FGLS (suponiendo que ya está definida en el código original)."""
    X = np.vstack([np.ones_like(x), x]).T
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
    residuals = y - X @ beta_hat
    weights = 1 / (x ** 2)
    W = np.diag(weights)
    beta_hat_wls = np.linalg.inv(X.T @ W @ X) @ (X.T @ W @ y)
    residuals_wls = y - X @ beta_hat_wls
    sigma_sq = np.sum(residuals_wls**2) / (len(y) - 2)
    var_beta = sigma_sq * np.linalg.inv(X.T @ W @ X)
    se_beta1_hat = np.sqrt(var_beta[1, 1])
    return beta_hat_wls[0], beta_hat_wls[1], se_beta1_hat

In [13]:
np.random.seed(7)
# Definir número de observaciones
n = 5  # Ajusta según necesidad

# Generar datos
x = (10 - 1) * np.random.uniform(size=n) + 1
u = np.random.normal(size=n) * x
print(x,u)

[1.6867746  8.01926913 4.94568308 7.5111866  9.80190561] [ 0.15715015 25.66899652 -0.77641616 -2.17923096 19.85642126]


In [14]:
np.random.seed(7)
x = np.random.uniform(1, 10, n)
u = np.random.normal(scale=x)
print(x,u)

[1.6867746  8.01926913 4.94568308 7.5111866  9.80190561] [ 0.15715015 25.66899652 -0.77641616 -2.17923096 19.85642126]


In [17]:
np.random.seed(7)
omega = np.diag([4, 9, 16, 25, 36])
u = np.random.multivariate_normal(mean=np.zeros(n), cov=omega)
u

array([-1.57784606,  1.22254885,  0.13128065, -2.32968685, 10.14315422])

In [7]:
x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
u = np.random.multivariate_normal(mean=np.zeros(10), cov=omega)  # u ~ N(0, omega)
y = beta_0_true + beta_1_true * x + u

ValueError: mean and cov must have same length

In [12]:
omega = np.diag([4, 9, 16, 25, 36][:10])
u = np.random.multivariate_normal(mean=np.zeros(10), cov=omega)
u

ValueError: mean and cov must have same length

In [8]:
u = np.random.multivariate_normal(mean=np.zeros(10), cov=omega)
u

ValueError: mean and cov must have same length

In [2]:
# Parámetros del modelo
beta_0 = -3
beta_1 = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas y covarianzas de u
n_observations = 5  # Tamaño de cada muestra
n_samples = 5000  # Número de muestras


# Generación de muestras
samples = []
np.random.seed(3649)  #Últimos 4 números de mi documento de identidad.

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1 * x + u  # Generar y
    samples.append(pd.DataFrame({'x': x, 'u': u, 'y': y}))

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples, keys=range(n_samples), names=['sample', 'index']).reset_index()

# Mostrar algunas filas
all_samples.head(10)

,sample,index,x,u,y
0,0,0,44.691948,0.394367,33.147926
1,0,1,33.945501,-2.739349,21.417052
2,0,2,37.364044,-5.117829,21.773406
3,0,3,37.251050,1.502358,28.303198
4,0,4,34.525174,6.038023,30.658162
5,1,0,3.409759,0.030947,-0.241246
6,1,1,14.352028,-0.504023,7.977600
7,1,2,30.112575,2.001504,23.091564
8,1,3,5.797610,3.095406,4.733495
9,1,4,20.579967,8.232626,21.696599


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5

x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
x2 = x**2
u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
y = beta_0_true + beta_1_true * x + u  # Generar y

# Creamos un DataFrame
df = pd.DataFrame({
    'x': x,
    'x2': x2,
    'u': u,
    'y': y
})

# 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
X = sm.add_constant(x)  # Matriz de diseño con intercepto
ols_model = sm.OLS(y, X)  # Estimación OLS
ols_results = ols_model.fit()  # Ajuste de OLS

# 2. Calcular los residuos del modelo y elevarlos al cuadrado
df['u_hat'] = ols_results.resid
df['u_hat2'] = df['u_hat']**2

# 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x x2, se estima por OLS usando u_hat2 como proxy de sigma2
ols_model_aux = sm.OLS(df['u_hat2'], df[['x', 'x2']]) #Ver de agregar intercepto.
ols_model_aux_results = ols_model_aux.fit()
gamma_hat1 = ols_model_aux_results.params['x']
gamma_hat2 = ols_model_aux_results.params['x2']

# 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes)
df['sigma2_hat'] = ols_model_aux_results.predict(df[['x', 'x2']])

# 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por la raíz de sigma2_hat y estimar por OLS
df['u_hat2_over_sigma2_hat'] = df['u_hat2'] / df['sigma2_hat']
df['x_over_sigma2_hat'] = df['x'] / df['sigma2_hat']
df['x2_over_sigma2_hat'] = df['x2'] / df['sigma2_hat']

ols_model_aux2 = sm.OLS(df['u_hat2_over_sigma2_hat'], df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']]) #Ver de agregar intercepto.
ols_model_aux_results2 = ols_model_aux2.fit()
gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

# 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
df['sigma2_tilde'] = ols_model_aux_results2.predict(df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']])
df['sigma2_tilde'] = np.sqrt(df['sigma2_tilde'])

# 7. Usar uno sobre sigma tilde como ponderador en la regresión de y sobre x
df['y_estrella'] = df['y'] / df['sigma_tilde']
df['x_estrella'] = df['x'] / df['sigma_tilde']

X_estrella = sm.add_constant(df['x_over_sigma_tilde']) 
final_ols_model = sm.OLS(df['y_estrella'], X_estrella)
final_ols_results = final_ols_model.fit() 

gamma_hat: 1.0190342558308039
beta1 (coeficiente de x): 1.049125141002301
beta0 (intercepto): 1.1894337904253731


In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
#n_samples = 5000
n_observations = 5

x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
x2 = x**2
u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
y = beta_0_true + beta_1_true * x + u  # Generar y

# Creamos un DataFrame
df = pd.DataFrame({
    'x': x,
    'x2': x2,
    'u': u,
    'y': y
})

# 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
X = sm.add_constant(x)  # Matriz de diseño con intercepto
ols_model = sm.OLS(y, X)  # Estimación OLS
ols_results = ols_model.fit()  # Ajuste de OLS

# 2. Calcular los residuos del modelo y elevarlos al cuadrado
df['u_hat'] = ols_results.resid
df['u_hat2'] = df['u_hat']**2

# 3. Regresión auxiliar: u_hat2 ~ x + x2 (incluyendo intercepto)
aux_X = sm.add_constant(df[['x', 'x2']])
ols_model_aux = sm.OLS(df['u_hat2'], aux_X)
ols_model_aux_results = ols_model_aux.fit()
gamma_hat1 = ols_model_aux_results.params['x']
gamma_hat2 = ols_model_aux_results.params['x2']

# 4. Usar las estimaciones para obtener las varianzas ajustadas
df['sigma2_hat'] = ols_model_aux_results.predict(aux_X)

# 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por la raíz de sigma2_hat y estimar por OLS
df['u_hat2_over_sigma2_hat'] = df['u_hat2'] / df['sigma2_hat']
df['x_over_sigma2_hat'] = df['x'] / df['sigma2_hat']
df['x2_over_sigma2_hat'] = df['x2'] / df['sigma2_hat']

aux_X_2 = sm.add_constant(df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']])
ols_model_aux2 = sm.OLS(df['u_hat2_over_sigma2_hat'], aux_X_2)
ols_model_aux_results2 = ols_model_aux2.fit()
gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

# 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
df['sigma2_tilde'] = ols_model_aux_results2.predict(aux_X_2)
df['sigma_tilde'] = np.sqrt(df['sigma2_tilde'])

# 7. Usar uno sobre sigma tilde como ponderador en la regresión de y sobre x
df['y_star'] = df['y'] / df['sigma_tilde']
df['x_star'] = df['x'] / df['sigma_tilde']

aux_X_star = sm.add_constant(df['x_star']) 
final_ols_model = sm.OLS(df['y_star'], aux_X_star)
final_ols_results = final_ols_model.fit() 

# Estimaciones finales
beta_0_hat = final_ols_results.params['const']
beta_1_hat = final_ols_results.params['x_star']
print(f"Estimated beta_0: {beta_0_hat:.4f}, beta_1: {beta_1_hat:.4f}")

Estimated beta_0: 19.1022, beta_1: 0.1914


Ejemplo de all_samples:
   sample          x          y         u
0       0  44.691948  33.147926  0.394367
1       0  33.945501  21.417052 -2.739349
2       0  37.364044  21.773406 -5.117829
3       0  37.251050  28.303198  1.502358
4       0  34.525174  30.658162  6.038023

Ejemplo de results_df:
   sample  beta_0_hat  beta_1_hat  se_beta1_hat
0       0   19.102219    0.191446      1.192612


In [15]:
results_df.head(1)

,sample,beta_0_hat,beta_1_hat,se_beta1_hat
0,0,19.102219,0.191446,1.192612


In [9]:
results_df['beta_1_hat'].median()

0.7429979282613803

In [11]:
all_samples.shape

(25000, 4)

In [10]:
results_df.shape

(5000, 3)

In [5]:
import numpy as np

# Definimos parámetros
beta_0 = -3
beta_1 = 0.8
omega = np.diag([4, 9, 16, 25, 36])
N = 5  # Observaciones por muestra
M = 5000  # Cantidad de muestras

# Generación de datos
np.random.seed(3649)
x = np.random.uniform(1, 50, size=(M, N))  # x ~ U[1, 50]
chol_omega = np.linalg.cholesky(omega)     # P = Cholesky de omega
u = chol_omega @ np.random.randn(N, M)    # u ~ N(0, omega)
u = u.T  # Transpuesta para mantener dimensiones MxN
y = beta_0 + beta_1 * x + u               # y_i = beta_0 + beta_1 * x_i + u_i

# Transformación para MCC
P_inv = np.linalg.inv(chol_omega)         # Inversa de P
y_star = y @ P_inv.T                      # y* = P^-1 * y
x_star = x @ P_inv.T                      # X* = P^-1 * X
x_star = np.hstack([np.ones((M, 1)), x_star])  # Agregar constante a X*

# Estimación MCC
beta_mcc = np.linalg.inv(x_star.T @ x_star) @ x_star.T @ y_star

# Resultados
print("Estimación por MCC (beta_0, beta_1):")
print(beta_mcc.mean(axis=0))  # Promedio de betas estimadas en todas las muestras

Estimación por MCC (beta_0, beta_1):
[-0.10411783 -0.04700603  0.00813422  0.03736534  0.06757768]


In [13]:
beta_mcc[0]

array([-1.41230844, -1.08667462, -0.74686156, -0.56526369, -0.38246838])

In [8]:
x[0]

array([44.69194777, 33.9455005 , 37.36404406, 37.25104984, 34.52517416])

In [11]:
np.hstack([np.ones((M, 1)), x]).shape

(5000, 6)

In [21]:
display(omega, chol_omega)

array([[ 4,  0,  0,  0,  0],
       [ 0,  9,  0,  0,  0],
       [ 0,  0, 16,  0,  0],
       [ 0,  0,  0, 25,  0],
       [ 0,  0,  0,  0, 36]])

array([[2., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0.],
       [0., 0., 4., 0., 0.],
       [0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 6.]])

In [1]:
import numpy as np
import pandas as pd

# Configuración inicial
np.random.seed(3649)

# Matriz omega (varianzas)
omega = np.diag([4, 9, 16, 25, 36])

# Descomposición de Cholesky de omega
P = np.linalg.cholesky(omega)

# Inversa de P
P_inv = np.linalg.inv(P)

In [4]:
P_inv

array([[0.5       , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.25      , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.2       , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.16666667]])

In [17]:
import numpy as np
import pandas as pd

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
n_observations = 5  # Observaciones por muestra
n_samples = 5000  # Número de muestras

# Matriz omega (varianzas)
omega = np.diag([4, 9, 16, 25, 36])

# Descomposición de Cholesky de omega
P = np.linalg.cholesky(omega)

# Inversa de P
P_inv = np.linalg.inv(P)

# Contenedores para almacenar muestras y estimadores
samples = []
results = []

# Simulación de M muestras
for sample_id in range(n_samples):
    # 1. Generar datos
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0_true + beta_1_true * x + u  # Generar y

    # Almacenar muestra
    samples.append(pd.DataFrame({'sample': i, 'x': x, 'y': y, 'u': u}))

    # 2. Transformar para MCG
    y_tilde = P_inv @ y
    X = np.column_stack((np.ones(n_observations), x))  # Matriz de diseño original
    X_tilde = P_inv @ X

    # 3. Ajustar modelo transformado con MCO
    beta_hat = np.linalg.inv(X_tilde.T @ X_tilde) @ (X_tilde.T @ y_tilde)

    # Almacenar resultados
    results.append([i, beta_0_hat, beta_1_hat, se_beta1_hat])

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples).reset_index(drop=True)

# Crear DataFrame con los resultados de FGLS
results_df = pd.DataFrame(results, columns=[
    'sample', 'beta_0_hat', 'beta_1_hat', 'se_beta1_hat'
])

# Mostrar ejemplos de los DataFrames generados
print("Ejemplo de all_samples:")
print(all_samples.head())
print("\nEjemplo de results_df:")
print(results_df.head())

NameError: name 'beta_0_hat' is not defined

In [19]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
n_observations = 5  # Observaciones por muestra
n_samples = 5000  # Número de muestras

# Matriz omega (varianzas)
omega = np.diag([4, 9, 16, 25, 36])

# Descomposición de Cholesky de omega
P = np.linalg.cholesky(omega)

# Inversa de P
P_inv = np.linalg.inv(P)

# Contenedores para almacenar muestras y estimadores
samples = []
results = []

# Simulación de M muestras
for i in range(n_samples):
    # 1. Generar datos
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0_true + beta_1_true * x + u  # Generar y

    # Almacenar muestra
    samples.append(pd.DataFrame({'sample': i, 'x': x, 'y': y, 'u': u}))

    # 2. Transformar para MCG
    y_tilde = P_inv @ y
    X = sm.add_constant(x)  # Agregar columna de unos automáticamente
    X_tilde = P_inv @ X

    # 3. Ajustar modelo transformado con statsmodels
    model = sm.OLS(y_tilde, X_tilde)
    results_model = model.fit()

    # Extraer coeficientes y error estándar
    beta_0_hat, beta_1_hat = results_model.params
    se_beta1_hat = results_model.bse[1]  # Error estándar del coeficiente beta_1

    # Almacenar resultados
    results.append([i, beta_0_hat, beta_1_hat, se_beta1_hat])

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples).reset_index(drop=True)

# Crear DataFrame con los resultados de FGLS
results_df = pd.DataFrame(results, columns=[
    'sample', 'beta_0_hat', 'beta_1_hat', 'se_beta1_hat'
])

# Mostrar ejemplos de los DataFrames generados
print("Ejemplo de all_samples:")
print(all_samples.head())
print("\nEjemplo de results_df:")
print(results_df.head())

Ejemplo de all_samples:
   sample          x          y         u
0       0  44.691948  33.147926  0.394367
1       0  33.945501  21.417052 -2.739349
2       0  37.364044  21.773406 -5.117829
3       0  37.251050  28.303198  1.502358
4       0  34.525174  30.658162  6.038023

Ejemplo de results_df:
   sample  beta_0_hat  beta_1_hat  se_beta1_hat
0       0  -11.213024    0.989465      0.301298
1       1   -3.104349    0.893636      0.119634
2       2    1.983310    0.630313      0.192682
3       3   -1.290987    0.670113      0.142292
4       4   -0.085546    0.737440      0.122292


In [20]:
results_df['beta_1_hat'].median()

0.7964059833013949

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(3639)
n_observations = 20

# Generar x1 y x2
x1_core = np.linspace(-1, 1, 18)  # 18 puntos entre -1 y 1
x1 = np.concatenate([[-1.1], x1_core, [1.1]])  # Agregar extremos -1.1 y 1.1
x2 = np.random.normal(0, 1, n_observations)
X = sm.add_constant(np.column_stack((x1, x2)))

# Coeficientes
beta0, beta1, beta2 = 1, 1, 1

#Errores
un = np.random.normal(0, 1, n_observations) #un: u normal.
v1 = np.ones(n_observations) #v1: v = 1
ut = np.random.standard_t(5, n_observations) #ut: u t-student.
v2 = np.exp(0.25 * x1 + 0.25 * x2) #v2: v = exp(0.25*x1 + 0.25*x2)

#Diseños
# Diseño 0: normalidad y homocedasticidad
y0 = beta0 + beta1 * x1 + beta2 * x2 + np.sqrt(v1) * un
# Diseño 1: normalidad y heterocedasticidad
y1 = beta0 + beta1 * x1 + beta2 * x2 + np.sqrt(v2) * un
# Diseño 2: t-student y heterocedasticidad
y3 = beta0 + beta1 * x1 + beta2 * x2 + np.sqrt(v2) * ut

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

np.random.seed(3639)
n_observations = 20
n_simulations = 5000

# Generar x1 y x2
x1_core = np.linspace(-1, 1, 18)  # 18 puntos entre -1 y 1
x1 = np.concatenate([[-1.1], x1_core, [1.1]])  # Agregar extremos -1.1 y 1.1
x2 = np.random.normal(0, 1, n_observations)
X = sm.add_constant(np.column_stack((x1, x2)))

# Coeficientes
beta0, beta1, beta2 = 1, 1, 1

# Test de White
significance_levels = [0.01, 0.05, 0.10]
rejection_counts = {alpha: 0 for alpha in significance_levels}

for _ in range(n_simulations):
    # Errores
    un = np.random.normal(0, 1, n_observations)
    v1 = np.ones(n_observations)
    
    # Diseño 0: normalidad y homocedasticidad
    y0 = beta0 + beta1 * x1 + beta2 * x2 + np.sqrt(v1) * un
    
    # Estimación por MCO
    modelo = sm.OLS(y0, X).fit()
    
    # Test de White
    white_test = het_white(modelo.resid, X)
    p_value = white_test[1]  # p-valor de la estadística LM
    
    # Contar rechazos de H0 para cada nivel de significancia
    for alpha in significance_levels:
        if p_value < alpha:
            rejection_counts[alpha] += 1

# Reportar tamaños del test
for alpha, count in rejection_counts.items():
    print(f"Tamaño del test al {int(alpha * 100)}%: {count / n_simulations:.4f}")

Tamaño del test al 1%: 0.0024
Tamaño del test al 5%: 0.0358
Tamaño del test al 10%: 0.0848


In [2]:
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

np.random.seed(3639)
n_simulaciones = 5000
n = 20

# Niveles de significancia
alpha_levels = [0.01, 0.05, 0.10]

# Contadores de rechazos
rechazos_diseño1 = {alpha: 0 for alpha in alpha_levels}
rechazos_diseño2 = {alpha: 0 for alpha in alpha_levels}

# Simulaciones
for _ in range(n_simulaciones):
    # Generar x1 y x2
    x1_core = np.linspace(-1, 1, 18)  # 18 puntos entre -1 y 1
    x1 = np.concatenate([[-1.1], x1_core, [1.1]])  # Agregar extremos -1.1 y 1.1
    x2 = np.random.normal(0, 1, n)

    # Generar errores poblacionales
    un = np.random.normal(0, 1, n)  # Normal(0,1)
    ut = np.random.standard_t(5, n)  # t-student con 5 grados de libertad
    v2 = np.exp(0.25 * x1 + 0.25 * x2)  # Heterocedasticidad

    # Diseños:
    u_diseño1 = np.sqrt(v2) * un  # Normalidad y heterocedasticidad
    u_diseño2 = np.sqrt(v2) * ut  # No-normalidad y heterocedasticidad

    # Aplicar test de White sobre los errores poblacionales
    X_aux = sm.add_constant(np.column_stack((x1, x2)))
    
    # Diseño 1
    white_test_1 = het_white(u_diseño1, X_aux)
    p_value_1 = white_test_1[1]  # p-valor del test de White

    # Diseño 2
    white_test_2 = het_white(u_diseño2, X_aux)
    p_value_2 = white_test_2[1]  # p-valor del test de White

    # Contar rechazos para cada nivel de significancia
    for alpha in alpha_levels:
        if p_value_1 < alpha:
            rechazos_diseño1[alpha] += 1
        if p_value_2 < alpha:
            rechazos_diseño2[alpha] += 1

# Calcular tasas de rechazo (poder del test)
poder_diseño1 = {alpha: rechazos_diseño1[alpha] / n_simulaciones for alpha in alpha_levels}
poder_diseño2 = {alpha: rechazos_diseño2[alpha] / n_simulaciones for alpha in alpha_levels}

# Resultados
print("Poder del test de White usando errores poblacionales:")
print(f"Diseño 1 (Normalidad y heterocedasticidad): {poder_diseño1}")
print(f"Diseño 2 (No-normalidad y heterocedasticidad): {poder_diseño2}")

Poder del test de White usando errores poblacionales:
Diseño 1 (Normalidad y heterocedasticidad): {0.01: 0.0142, 0.05: 0.0894, 0.1: 0.165}
Diseño 2 (No-normalidad y heterocedasticidad): {0.01: 0.0194, 0.05: 0.0888, 0.1: 0.148}


In [1]:
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

np.random.seed(3639)
n_simulaciones = 5000
n = 20

# Niveles de significancia
alpha_levels = [0.01, 0.05, 0.10]

# Contadores de rechazos
rechazos_diseño1 = {alpha: 0 for alpha in alpha_levels}
rechazos_diseño2 = {alpha: 0 for alpha in alpha_levels}

# Simulaciones
for _ in range(n_simulaciones):
    # Generar x1 y x2
    x1_core = np.linspace(-1, 1, n-2)  # 18 puntos entre -1 y 1
    x1 = np.concatenate([[-1.1], x1_core, [1.1]])  # Agregar extremos -1.1 y 1.1
    x2 = np.random.normal(0, 1, n)

    # Generar errores poblacionales
    un = np.random.normal(0, 1, n)  # Normal(0,1)
    ut = np.random.standard_t(5, n)  # t-student con 5 grados de libertad
    v2 = np.exp(0.25 * x1 + 0.25 * x2)  # Heterocedasticidad

    # Diseños:
    u_diseño1 = np.sqrt(v2) * un  # Normalidad y heterocedasticidad
    u_diseño2 = np.sqrt(v2) * ut  # No-normalidad y heterocedasticidad

    # Aplicar test de White sobre los errores poblacionales
    X_aux = sm.add_constant(np.column_stack((x1, x2)))
    
    # Diseño 1
    white_test_1 = het_white(u_diseño1, X_aux)
    p_value_1 = white_test_1[1]  # p-valor del test de White

    # Diseño 2
    white_test_2 = het_white(u_diseño2, X_aux)
    p_value_2 = white_test_2[1]  # p-valor del test de White

    # Contar rechazos para cada nivel de significancia
    for alpha in alpha_levels:
        if p_value_1 < alpha:
            rechazos_diseño1[alpha] += 1
        if p_value_2 < alpha:
            rechazos_diseño2[alpha] += 1

# Calcular tasas de rechazo (poder del test)
poder_diseño1 = {alpha: rechazos_diseño1[alpha] / n_simulaciones for alpha in alpha_levels}
poder_diseño2 = {alpha: rechazos_diseño2[alpha] / n_simulaciones for alpha in alpha_levels}

# Resultados
print("Poder del test de White usando errores poblacionales:")
print(f"Diseño 1 (Normalidad y heterocedasticidad): {poder_diseño1}")
print(f"Diseño 2 (No-normalidad y heterocedasticidad): {poder_diseño2}")

Poder del test de White usando errores poblacionales:
Diseño 1 (Normalidad y heterocedasticidad): {0.01: 0.0142, 0.05: 0.0894, 0.1: 0.165}
Diseño 2 (No-normalidad y heterocedasticidad): {0.01: 0.0194, 0.05: 0.0888, 0.1: 0.148}


In [22]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS, add_constant

# Fijar semilla para reproducibilidad
np.random.seed(144)

# Cantidad de observaciones
n_obs = 20

# Generar las variables
x1 = np.linspace(-1.1, 0.9, n_obs)
x2 = np.random.normal(size=n_obs)
x0 = np.ones(n_obs)

# Crear DataFrame
data = pd.DataFrame({"x0": x0, "x1": x1, "x2": x2})

# Generar la matriz X y X_2
X = data[["x0", "x1", "x2"]].values
X_2 = data[["x1", "x2", "x0"]].values

# Generar las variables v, u y y
v = np.exp(0.25 * x1 + 0.25 * x2)
u = np.random.normal(0, 1, size=n_obs)
y = 1 + x1 + x2 + np.sqrt(v) * u

data["v"] = v
data["u"] = u
data["y"] = y

# Regresión: Incluimos explícitamente la constante
model = OLS(y, X)  # Usamos X completo, incluyendo x0
results = model.fit()

# Obtener los parámetros estimados (B)
B = results.params  # Esto ahora tendrá 3 elementos (x0, x1, x2)

# Generar la matriz de omega de White
Y = y.reshape(-1, 1)
uhat = Y - X_2 @ B.reshape(-1, 1)  # Ahora X_2 y B tienen dimensiones compatibles

# Continuar el cálculo como estaba
data["uhat"] = uhat.flatten()
data["uhat_2"] = data["uhat"] ** 2
uhat_2 = np.diag(data["uhat_2"].values)

omega_hat = uhat_2

# Matriz de omega de White con residuos originales
u_ori = (np.sqrt(v) * u) ** 2
data["u_ori"] = u_ori
omega_wori = np.diag(u_ori)

# Matrices de varianzas y covarianzas de los betas
X_transpose_X_inv = np.linalg.inv(X.T @ X)

Sigma = X_transpose_X_inv @ (X.T @ omega @ X) @ X_transpose_X_inv
Sigma_hat = X_transpose_X_inv @ (X.T @ omega_hat @ X) @ X_transpose_X_inv
Sigma_wori = X_transpose_X_inv @ (X.T @ omega_wori @ X) @ X_transpose_X_inv

# Sesgos relativos
bias_B0 = (Sigma_hat[0, 0] - Sigma[0, 0]) / Sigma[0, 0]
bias_B1 = (Sigma_hat[1, 1] - Sigma[1, 1]) / Sigma[1, 1]
bias_B2 = (Sigma_hat[2, 2] - Sigma[2, 2]) / Sigma[2, 2]

# Sesgos relativos con White con residuos originales
bias_ori_B0 = (Sigma_wori[0, 0] - Sigma[0, 0]) / Sigma[0, 0]
bias_ori_B1 = (Sigma_wori[1, 1] - Sigma[1, 1]) / Sigma[1, 1]
bias_ori_B2 = (Sigma_wori[2, 2] - Sigma[2, 2]) / Sigma[2, 2]

# Imprimir resultados
print("Sesgos relativos estimados (White):")
print(f"B0: {bias_B0}, B1: {bias_B1}, B2: {bias_B2}")

print("\nSesgos relativos estimados (White con residuos originales):")
print(f"B0: {bias_ori_B0}, B1: {bias_ori_B1}, B2: {bias_ori_B2}")

Sesgos relativos estimados (White):
B0: -0.10264479591532095, B1: 0.07860651184160447, B2: -0.3352494413374326

Sesgos relativos estimados (White con residuos originales):
B0: -0.19168155137200596, B1: 0.02866193652607788, B2: -0.5327384396207882


In [ ]:
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS
def fgls_estimation(x, y, omega):
    W = np.linalg.inv(omega)  # Matriz de pesos inversos
    X = np.column_stack((np.ones(len(x)), x))  # Matriz de diseño
    beta_hat = np.linalg.inv(X.T @ W @ X) @ (X.T @ W @ y)  # Estimación FGLS
    var_beta = np.linalg.inv(X.T @ W @ X)  # Varianza de los estimadores
    return beta_hat, np.sqrt(np.diag(var_beta))  # Estimadores y errores estándar

# Simulaciones
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    beta_hat, se_beta = fgls_estimation(x, y, omega)  # Estimación FGLS
    t_stat = (beta_hat[1] - beta_1_true) / se_beta[1]  # Estadístico t para beta_1
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))  # Valor p
    results.append([beta_hat[0], beta_hat[1], se_beta[0], se_beta[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        beta_hat, se_beta = fgls_estimation(x, y, omega)
        t_stat = (beta_hat[1] - beta_1_true) / se_beta[1]
        p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.0, 0.05: 0.0012}
Poder del test: {0: 0.9014, 0.4: 0.452}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -2.958325  0.798089
median -2.890889  0.796406
std     4.649156  0.162026


In [ ]:
import statsmodels.api as sm
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS usando statsmodels
def fgls_estimation(x, y, omega):
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    model = sm.GLS(y, X, sigma=omega)  # Estimación FGLS con sigma = omega
    results = model.fit()  # Ajustar el modelo
    return results

# Simulaciones
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    result = fgls_estimation(x, y, omega)  # Estimación FGLS
    t_stat = result.tvalues[1]  # Estadístico t para beta_1
    p_value = result.pvalues[1]  # Valor p para beta_1
    results.append([result.params[0], result.params[1], result.bse[0], result.bse[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        result = fgls_estimation(x, y, omega)
        t_stat = result.tvalues[1]
        p_value = result.pvalues[1]
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.6018, 0.05: 0.8908}
Poder del test: {0: 0.0532, 0.4: 0.5276}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -2.958325  0.798089
median -2.890889  0.796406
std     4.649156  0.162026


In [10]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

# Parámetros
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_observations = 5  # Pequeña muestra
n_samples = 5000  # Número de simulaciones

# Generar datos
np.random.seed(42)

def generate_data():
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    return x, y

# Método 1: Estimación con FGLS usando statsmodels
def fgls_statsmodels(x, y):
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    model = sm.GLS(y, X, sigma=omega)  # Estimación FGLS con sigma = omega
    results = model.fit()  # Ajustar el modelo
    return results

# Método 2: Estimación manual FGLS
def fgls_manual(x, y, omega):
    X = np.column_stack([np.ones(n_observations), x])  # Matriz de diseño con intercepto
    # Calcular la inversa de Omega
    omega_inv = np.linalg.inv(omega)
    # Estimar Beta usando la fórmula FGLS
    beta_hat = np.linalg.inv(X.T @ omega_inv @ X) @ (X.T @ omega_inv @ y)
    # Estimación de errores estándar (desviación estándar)
    residuals = y - X @ beta_hat
    sigma_hat = (residuals.T @ omega_inv @ residuals) / (n_observations - 2)  # Varianza de los errores
    cov_beta_hat = np.linalg.inv(X.T @ omega_inv @ X) * sigma_hat  # Varianza de los coeficientes
    se_beta_hat = np.sqrt(np.diag(cov_beta_hat))  # Desviación estándar de los coeficientes
    return beta_hat, se_beta_hat

# Comparación de ambos métodos
results_fgls_sm = []
results_fgls_manual = []

for _ in range(n_samples):
    x, y = generate_data()
    
    # Método 1: FGLS con statsmodels
    result_sm = fgls_statsmodels(x, y)
    results_fgls_sm.append([result_sm.params[0], result_sm.params[1], result_sm.bse[0], result_sm.bse[1]])
    
    # Método 2: Estimación manual
    beta_hat_manual, se_beta_hat_manual = fgls_manual(x, y, omega)
    results_fgls_manual.append([beta_hat_manual[0], beta_hat_manual[1], se_beta_hat_manual[0], se_beta_hat_manual[1]])

# Convertir resultados a DataFrame para comparar
results_fgls_sm_df = pd.DataFrame(results_fgls_sm, columns=['beta_0_sm', 'beta_1_sm', 'se_beta_0_sm', 'se_beta_1_sm'])
results_fgls_manual_df = pd.DataFrame(results_fgls_manual, columns=['beta_0_manual', 'beta_1_manual', 'se_beta_0_manual', 'se_beta_1_manual'])

# Comparar medias, medianas y desviaciones estándar
comparison = pd.concat([
    results_fgls_sm_df[['beta_0_sm', 'beta_1_sm', 'se_beta_0_sm', 'se_beta_1_sm']].agg(['mean', 'median', 'std']),
    results_fgls_manual_df[['beta_0_manual', 'beta_1_manual', 'se_beta_0_manual', 'se_beta_1_manual']].agg(['mean', 'median', 'std'])
], axis=1)

comparison

,beta_0_sm,beta_1_sm,se_beta_0_sm,se_beta_1_sm,beta_0_manual,beta_1_manual,se_beta_0_manual,se_beta_1_manual
mean,-3.124395,0.803626,3.777375,0.136698,-3.124395,0.803626,3.777375,0.136698
median,-3.050941,0.804804,3.135215,0.118106,-3.050941,0.804804,3.135215,0.118106
std,4.543793,0.162838,2.673090,0.088475,4.543793,0.162838,2.673090,0.088475


In [12]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS usando statsmodels con errores robustos
def fgls_estimation(x, y, omega):
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    model = sm.GLS(y, X, sigma=omega)  # Estimación FGLS con sigma = omega
    results = model.fit()  # Ajustar el modelo
    robust_results = results.get_robustcov_results(cov_type='HC3')  # Errores estándar robustos (HC3)
    return robust_results

# Simulaciones
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    robust_results = fgls_estimation(x, y, omega)  # Estimación FGLS con errores robustos
    t_stat = robust_results.tvalues[1]  # Estadístico t para beta_1
    p_value = robust_results.pvalues[1]  # Valor p para beta_1
    results.append([robust_results.params[0], robust_results.params[1], robust_results.bse[0], robust_results.bse[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        robust_results = fgls_estimation(x, y, omega)
        t_stat = robust_results.tvalues[1]
        p_value = robust_results.pvalues[1]
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.298, 0.05: 0.6176}
Poder del test: {0: 0.0282, 0.4: 0.2734}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -2.958325  0.798089
median -2.890889  0.796406
std     4.649156  0.162026


In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Parámetros del modelo
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas (heterocedasticidad)
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para realizar la estimación FGLS con errores robustos de White
def fgls_white_estimation(x, y, omega):
    # Paso 1: Estimación OLS inicial
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    ols_model = sm.OLS(y, X)  # Estimación OLS
    ols_results = ols_model.fit()  # Ajuste de OLS
    
    # Paso 2: Estimación de la matriz de varianzas-covarianzas de los errores
    residuals = ols_results.resid  # Residuos de la estimación OLS
    # Usamos la matriz de varianzas-covarianzas robusta (White)
    robust_cov = ols_results.get_robustcov_results(cov_type='HC3').cov_params()  # Matriz robusta de varianzas-covarianzas
    
    # Paso 3: Estimación FGLS con la matriz robusta
    W = np.linalg.inv(robust_cov)  # Matriz de pesos (inversa de la matriz de varianzas-covarianzas)
    fgls_model = sm.GLS(y, X, sigma=W)  # Estimación FGLS usando la matriz robusta
    fgls_results = fgls_model.fit()  # Ajuste FGLS
    
    return fgls_results

# Simulación de los datos
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    fgls_result = fgls_white_estimation(x, y, omega)  # Estimación FGLS White
    
    # Obtener estadísticos de interés
    t_stat = fgls_result.tvalues[1]  # Estadístico t para beta_1
    p_value = fgls_result.pvalues[1]  # Valor p para beta_1
    results.append([fgls_result.params[0], fgls_result.params[1], fgls_result.bse[0], fgls_result.bse[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        fgls_result = fgls_white_estimation(x, y, omega)
        p_value = fgls_result.pvalues[1]
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

ValueError: Sigma must be a scalar, 1d of length 5 or a 2d array of shape 5 x 5